In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from IPython.core.display import display, HTML

HTML("""
    <style>
    .jp-Stdin-input {
        width: 80% !important;
        height : 1.5em !important;
        font-size : 1em;
        background : white;
        border:1px solid #cccccc;
    }
    </style>
    """)

In [ ]:
import numpy as np
import torch
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
np.set_printoptions(suppress=True)
from Utils import *

In [ ]:
torch.cuda.set_device(0)

In [ ]:
import sys
sys.path.insert(0, 'preprocess/')
import vectorizer

import pickle

In [ ]:
vec = pickle.load(open('preprocess/sim_data.p', 'rb'))
add_frequencies(vec, vec.seq_text['train'])

In [ ]:
import model.Attn_Word_Pert as M
Model = M.Model

In [ ]:
X, Xt = vec.seq_text['train'], vec.seq_text['test']
y, yt = vec.label['train'], vec.label['test']

In [ ]:
from sklearn.metrics import classification_report, f1_score

def train(name='') :
    model = Model(vec.vocab_size, vec.word_dim, 64, dirname='sim', hidden_size=6)
    best_f1 = 0.0
    for i in tqdm_notebook(range(30)) :
        loss = model.train(X, y)
        o, he = model.evaluate(Xt)
        o = np.array(o)
        rep = classification_report(yt, (o > 0.5))
        f1 = f1_score(yt, (o > 0.5), pos_label=1)
        print(rep)
        stmt = '%s, %s' % (i, loss)
        if True : #f1 > best_f1 :
            best_f1 = f1
            dirname = model.save_values(add_name=name, save_model=True)
            print("Model Saved", f1)
        else :
            dirname = model.save_values(add_name=name, save_model=False)
            print("Model not saved", f1)
        f = open(dirname + '/epoch.txt', 'a')
        f.write(stmt + '\n')
        f.write(rep + '\n')
        f.close()
    
    return model

In [ ]:
for i in range(20) :
    train(name='TEST_sim_exp_' + str(i))

In [ ]:
import os
dirs = os.listdir('outputs/attn_word_sim/')
dirs = [d for d in dirs if 'exp' in d]

In [ ]:
for e in dirs :
    model = load_model('outputs/attn_word_sim/' + e)
    yt_hat, attn_hat = evaluate_and_print(model, Xt, yt)

# **EVALUATION**

In [ ]:
def load_model(dirname) :
    model = Model(vec.vocab_size, vec.word_dim, 32, dirname='sst', hidden_size=6)
    model.dirname = dirname
    model.load_values(dirname)
    return model

In [ ]:
model = load_model('outputs/attn_word_sim/FriOct1913:14:512018_TEST_sim_exp_2/')

In [ ]:
yt_hat, attn_hat = evaluate_and_print(model, Xt, yt)

In [ ]:
plot_entropy(Xt, attn_hat)

In [ ]:
n = 4
print_attn(vec.map2words(Xt[n]), attn_hat[n])

# __Sampling__

In [ ]:
model.vec = vec
sampled_output = model.sampling_top(Xt, sample_vocab=100, topnum=5)

In [ ]:
import pickle
pickle.dump(sampled_output, open(model.dirname + '/sampled.p', 'wb'))

In [ ]:
sampled_output = pickle.load(open(model.dirname + '/sampled.p', 'rb'))

In [ ]:
generate_medians_from_sampling_top(sampled_output, attn_hat, yt_hat)

In [ ]:
distractors = get_distractors(sampled_output, attn_hat)
print_few_distractors(vec, Xt, attn_hat, sampled_output, distractors)

**Gradients**
=============

In [ ]:
grads = model.gradient_mem(Xt)
process_grads(grads)

In [ ]:
plot_grads(Xt, attn_hat, grads)

**Permutation**
===========

In [ ]:
perms = model.permute_attn(Xt)

In [ ]:
plot_permutations(perms, Xt, yt_hat, attn_hat)

**Adversary**
=============

In [ ]:
adversarial_outputs = model.adversarial(Xt, _type='uniform')
ad_y, ad_attn = adversarial_outputs

In [ ]:
jds = plot_adversarial(Xt, yt_hat, attn_hat, adversarial_outputs)

In [ ]:
idx = list(np.where(np.logical_and(np.array(jds) > 0.5, yt_hat > 0.7))[0])[:30]
idx

In [ ]:
n = 734
print_adversarial_example(vec.map2words(X[n]), attn_hat[n], ad_attn[n])
print(yt_hat[n], ad_y[n])

**Copy Runs**
=============

In [ ]:
copy_runs = model.copy_H_run(Xt)

In [ ]:
copy_outputs, copy_attn, copy_H_diff = copy_runs

In [ ]:
plot_attn_diff(Xt, attn_hat, copy_attn, "Attention vs Copy Attention")
plot_y_diff(X, attn_hat, yt_hat, copy_outputs)
plot_attn_diff(Xt, attn_hat, copy_H_diff, "Attention vs H difference")

**Zero Runs**
=============

In [ ]:
zero_runs = model.zero_H_run(Xt)

In [ ]:
zero_outputs, zero_H_diff = zero_runs

In [ ]:
plot_attn_diff(Xt, attn_hat, zero_H_diff)
plot_y_diff(X, attn_hat, yt_hat, zero_outputs)

**Remove and Run**
==================

In [ ]:
remove_outputs = model.remove_and_run(Xt)

In [ ]:
plot_y_diff(Xt, attn_hat, yt_hat, remove_outputs)

**Perturbation**
================

In [ ]:
perturb_outputs = model.perturbation_embedding(Xt)

In [ ]:
plot_pertub_embedding(Xt, attn_hat, yt_hat, perturb_outputs)